

## **MCP Architecture (High-Level Visual & Concepts)**

![Image](https://static.wixstatic.com/media/e6ecb9_c4f0699a79544b189988da9e705afecb~mv2.jpg/v1/fill/w_980%2Ch_619%2Cal_c%2Cq_85%2Cusm_0.66_1.00_0.01%2Cenc_avif%2Cquality_auto/e6ecb9_c4f0699a79544b189988da9e705afecb~mv2.jpg)

![Image](https://www.descope.com/_next/image?q=75\&url=https%3A%2F%2Fimages.ctfassets.net%2Fxqb1f63q68s1%2F6R2RtSw84mTFLKfYBPpqNQ%2Ff1ef779c252dde2997f6cc1ab92fa794%2FMCP_general_architecture-min.png\&w=1920)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1358/1%2AsSVwjNeVLHzZ6wou1TAtTg.png)

![Image](https://www.tenable.com/sites/default/files/images/blog/90a9242f-116e-4a45-89d6-edb138f3c25f.png)

![Image](https://workato.com/the-connector/wp-content/uploads/2025/04/AD_4nXeVNy0WC-VcqhWQGgZ9NsWtodJ7BM_n6WRUs58hQEAcNvCtrdtmNBr7_HYDK47TiZt0xM3uBA-14Fc0B4I-xpC8qotMD2LXPFBVIEZFkDfmhDRlvCOPCwCeXiAFNtcOEdx_zKTjkeyZfP96ADPBWnGE3_T44ny1JMJ.png)

The **Model Context Protocol (MCP)** is a **standard, JSON-RPC based client-server protocol** that lets LLM applications (Clients/Hosts) discover and invoke external tools without embedding custom integrations per tool. It solves the *N×M maintenance problem* by isolating **tools on servers** and exposing them through a consistent interface. ([Model Context Protocol][1])

* **Clients/Hosts**: LLM workflows (LangGraph, agents, chatbots).
* **Servers**: Standalone services exposing tools (math, expenses, APIs, data stores).
* **Transport**: stdio, HTTP/SSE, socket JSON-RPC.
* **Protocol**: JSON-RPC for method invocation, tool discovery, tool metadata, input validation. ([LangChain Docs][2])

Below the surface, MCP resembles a “**USB-C port for AI tools**”: standardized at the protocol layer so any server can plug into any client. ([Medium][3])

---

## **Async Programming Fundamentals for MCP (Python)**

MCP clients and servers require true **asynchronous IO**—the ability to do non-blocking waits and concurrency in a single thread using **async/await** (Python 3.7+). The async model runs an **event loop**, scheduling coroutines rather than blocking threads. ([Real Python][4])

![Image](https://geekpython.in/wp-content/uploads/2023/08/async-await.png)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/1%2AE_hYZrM8bccKFecUDlD8ng.png)

![Image](https://cdn.hackersandslackers.com/2021/04/async_eventloop.jpg)

Key Points:

* **Event loop**: Schedules tasks and resumes them when IO completes. ([Real Python][4])
* **async/await**: `async` declares a coroutine; `await` yields control until awaited task completes. ([Real Python][4])
* Use **async frameworks** (FastAPI, trio, asyncio) for servers/clients.

**Python async snippet (blocking vs async)**

```python
import asyncio

async def slow_io(x: int) -> str:
    # simulate network delay
    await asyncio.sleep(1)
    return f"done:{x}"

async def main():
    # schedule coroutines concurrently
    results = await asyncio.gather(slow_io(1), slow_io(2))
    print(results)

asyncio.run(main())
```

---

## **Core Implementation Blueprint with LangGraph + MCP**

### **1. Install Dependencies**

```bash
python3 -m venv venv && . venv/bin/activate
pip install langchain langgraph langchain-mcp-adapters[all] fastmcp uvicorn
```

Pin versions in `requirements.txt`:

```
langchain==0.1.0
langgraph==0.1.0
langchain-mcp-adapters==0.1.0
fastmcp==0.1.0
uvicorn==0.23.0
```

Ensure TLS1.3+ in deployment (nginx/CLOUDFLARE + Uvicorn with SSL certs).

---

### **2. MCP Server (FastAPI + FastMCP)**

This server exposes two tools (`add` and `subtract`) via MCP.

```python
# mcp_server.py
import asyncio
from fastmcp import FastMCP, tool

mcp = FastMCP()

@tool()
async def add(a: float, b: float) -> float:
    return a + b

@tool()
async def subtract(a: float, b: float) -> float:
    return a - b

if __name__ == "__main__":
    mcp.run(transport="stdio")  # JSON-RPC over stdio
```

* Tools are decorated with `@tool()` and use **async defs**.
* Run via `python mcp_server.py`. This is the **local server** for MCP.
* **Security**: Validate all inputs, enforce rate limits, and sign manifests for tool definitions (see academic threat models). ([arXiv][5])

---

### **3. LangGraph App (Client + MCP Integration)**

Integrate MCP tools into a LangGraph workflow:

```python
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph

# multi-server client
client = MultiServerMCPClient()
client.add_client(
    name="math_server",
    transport="stdio",
    command="python",
    args=["./mcp_server.py"]
)

async def build_graph(llm):
    # connect to MCP servers
    await client.connect()
    mcp_tools = await client.get_tools()
    
    # bind tools to LLM
    llm_with_tools = llm.bind_tools(mcp_tools)
    
    async def chat_node(state):
        # tool invocation handled via MCP
        return {"messages": [await llm_with_tools.ainvoke(state["messages"])]}

    graph = StateGraph(State)
    graph.add_node("chat", chat_node)
    workflow = graph.compile()
    return workflow

async def main():
    llm = ...  # initialize async LLM
    app = await build_graph(llm)
    result = await app.ainvoke({"messages": [("user","Add 2 + 3")]})
    print(result)

asyncio.run(main())
```

Benefits:

* **No hardcoded tool implementations**: The LLM binds tools dynamically. ([LangChain Docs][2])
* **Async end-to-end**: All nodes use `await` and play nicely with the event loop. ([Real Python][4])

---

## **Production Deployment & Hardening**

* **Backend**: FastAPI + Uvicorn + HTTPS (backend).
* **Frontend**: React/Next.js with SSO/MFA.
* **CI/CD**:

  * Build + test MCP servers in GitHub Actions.
  * Lint + type checks (mypy).
  * Canary deploy + rollback.
* **Observability**: Tracing (Opentelemetry), logs (structured JSON), SLOs for tool latency.
* **Trust Boundaries**:

  * Validate and sanitize all MCP inputs/outputs.
  * PCI/PHI compliance if needed.
  * Signature verificaiton of tool definitions to mitigate descriptor poisoning (see threat model). ([arXiv][5])

---

## **Interview-Level MCP + LangGraph Q&A**

**Q:** *Why use MCP instead of built-in tools?*
**A:** MCP standardizes integration so you avoid brittle, tightly coupled code; only servers need updates when APIs change. ([Model Context Protocol][1])

**Q:** *Why must Python code be asynchronous for MCP?*
**A:** MCP clients use async transports and event loops to manage concurrent tool calls without blocking. ([Real Python][4])

**Q:** *How do you mitigate security risks like tool abuse or descriptor poisoning?*
**A:** Sign tool manifests, vet descriptors in a CI pipeline, and limit invocation scopes. Academic research identifies classes of semantic attacks (e.g., tool poisoning) requiring layered defenses. ([arXiv][5])

**Q:** *How to combine standard LangChain tools with MCP tools?*
**A:** Fetch MCP tools and merge them into your LangChain tool list; LLM will treat them uniformly.

**Q:** *How to integrate RAG with MCP workflows?*
**A:** Add a vector database connector as an MCP server or incorporate RAG in the LLM graph before tool binding.

---




---

# 🧩 **What MCP Solves**

Standard LangChain/LangGraph tools are **embedded inside the application**, which creates two scaling failures:

---

## **1. API Change Failure**

If an external service changes its API, such as:

```
/repos/{owner}/{repo}/pulls → /repos/{owner}/{repo}/prs
```

then every chatbot using that tool breaks because the API logic is inside the chatbot codebase.

---

## **2. $N \times M Maintenance Failure**

If you have:

* **M chatbots** = 5
* **N tools/services** = 10

Then you maintain:

```
5 × 10 = 50 integration points
```

because each chatbot integrates each tool individually.

MCP solves this by:

* moving tool logic to a **Server**
* letting chatbots act as **Clients**
* standardizing the interface between them

So updates happen once, centrally, not in every client.

---

# ⚙️ **Code Comparison**

Below are the two models solving the **same problem**: retrieving GitHub PRs.

---

# ❌ **Without MCP (Hardcoded / Brittle)**

Tool logic lives inside the chatbot application.

```python
@tool
def get_pull_requests(owner: str, repo: str):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
    headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
    response = requests.get(url, headers=headers)
    return response.json()

llm_with_tools = llm.bind_tools([get_pull_requests])
```

**Characteristics:**

| Property       | Result              |
| -------------- | ------------------- |
| Tool logic     | embedded in chatbot |
| API versioning | breaks app          |
| Auth & errors  | your responsibility |
| Sharing tools  | copy/paste          |
| Deployment     | tightly coupled     |
| Scaling        | poor                |

If the GitHub API changes, all chatbot apps must be patched and redeployed.

---

# ✅ **With MCP (Decoupled / Configurable)**

Tool logic lives in a **GitHub MCP Server**. Chatbot only consumes tools.

```python
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient()

client.add_client(
    name="github_server",
    transport="stdio",
    command="python",
    args=["./github_mcp_server.py"]
)

async def build_graph():
    await client.connect()

    mcp_tools = await client.get_tools()  # auto-discover tools

    llm_with_tools = llm.bind_tools(mcp_tools)
```

**Characteristics:**

| Property      | Result              |
| ------------- | ------------------- |
| Tool logic    | external server     |
| API changes   | server updates only |
| Auth & errors | encapsulated        |
| Sharing tools | plug-and-play       |
| Deployment    | loosely coupled     |
| Scaling       | excellent           |

The chatbot doesn’t care if the server uses REST, GraphQL, gRPC, v1 or v2.

---

# 🧱 **System Roles**

| Component      | Responsibility                  |
| -------------- | ------------------------------- |
| **MCP Server** | implements tools (GitHub logic) |
| **MCP Client** | discovers + invokes tools       |
| **LLM**        | decides when to call tools      |

This separation mirrors modern microservice architecture:

```
LLM (client) → MCP → Tool Server (service)
```

---

# ♻️ **Reusability Benefit**

The **same GitHub MCP server** can be attached to:

* LangGraph chatbot
* Claude Desktop
* VSCode
* Cursor IDE
* future MCP-compatible hosts

without rewriting tool integrations.

---

# 🏗️ **Why Async is Required**

MCP uses asynchronous IO for:

* server subprocess communication (stdio / SSE)
* tool discovery
* tool invocation
* streaming outputs

Therefore, LangGraph nodes must use:

```python
async def ...
await ...
```

---

# 🚨 **Streamlit Note**

The transcript correctly notes:

* Streamlit = synchronous
* MCP = asynchronous

Result: requires “event-loop hacks”
Better production stack:

* **Backend:** FastAPI (async-native)
* **Frontend:** Next.js / React

---

# ⛓️ **Conclusion**

MCP transforms tool usage from:

```
embedded logic → external service
```

yielding:

✓ lower coupling
✓ centralized updates
✓ better scale
✓ interoperability across clients
✓ version independence

---





---

# ✔️ **Case 1 — WITHOUT MCP (classic LangChain tool)**

Here the math logic is **embedded inside the app**.

### **math_tool.py**

```python
async def add(a: float, b: float) -> float:
    return a + b
```

### **app_without_mcp.py**

```python
import asyncio
from langchain.tools import StructuredTool
from langgraph.graph import StateGraph, START
from langchain.chat_models import ChatOpenAI

class State(dict):
    pass

# define local tool
add_tool = StructuredTool.from_function(add)

async def main():
    llm = ChatOpenAI(model="gpt-4o-mini")
    llm_with_tools = llm.bind_tools([add_tool])

    async def chat(state: State):
        out = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": [out]}

    graph = StateGraph(State)
    graph.add_node("chat", chat)
    graph.add_edge(START, "chat")
    app = graph.compile()

    result = await app.ainvoke({
        "messages": [("user","add 4 and 7")]
    })
    print(result)

asyncio.run(main())
```

### **Output**

```
{'messages': [{'role': 'assistant', 'content': '11'}]}
```

---

# ❌ **Problems “WITHOUT MCP”**

* Tool logic lives inside the app
* If API changes → you must edit app code
* If 10 apps need the tool → copy/paste hell
* No versioning or discovery
* No remote tools (only local Python)

---

# ✔️ **Case 2 — WITH MCP (standard protocol)**

Now math logic moves to a **separate MCP server**.

### **mcp_math_server.py**

```python
from fastmcp import FastMCP, tool

mcp = FastMCP()

@tool()
async def add(a: float, b: float) -> float:
    return a + b

if __name__ == "__main__":
    mcp.run(transport="stdio")
```

MCP server is standalone. Can run:

```bash
python mcp_math_server.py
```

---

### **app_with_mcp.py**

```python
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, START
from langchain.chat_models import ChatOpenAI

class State(dict):
    pass

async def main():
    llm = ChatOpenAI(model="gpt-4o-mini")
    client = MultiServerMCPClient()

    client.add_client(
        name="math",
        transport="stdio",
        command="python",
        args=["./mcp_math_server.py"]
    )

    await client.connect()
    tools = await client.get_tools()
    llm_with_tools = llm.bind_tools(tools)

    async def chat(state: State):
        out = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": [out]}

    graph = StateGraph(State)
    graph.add_node("chat", chat)
    graph.add_edge(START, "chat")
    app = graph.compile()

    result = await app.ainvoke({
        "messages": [("user","add 4 and 7")]
    })
    print(result)

asyncio.run(main())
```

### **Output**

```
{'messages': [{'role': 'assistant', 'content': '11'}]}
```

---

# 🧩 **Side-by-Side Summary**

| Feature            | Without MCP     | With MCP            |
| ------------------ | --------------- | ------------------- |
| Where tool lives   | inside app      | separate server     |
| Tool update cost   | modify app code | update server only  |
| Reuse across apps  | poor            | excellent           |
| Versioning         | manual          | built-in            |
| Discovery          | none            | yes (`get_tools()`) |
| Transport          | function call   | JSON-RPC stdio/SSE  |
| Remote support     | no              | yes                 |
| Multiple tools     | copy/paste      | auto discover       |
| Standard Interface | no              | yes (MCP)           |
| DevOps             | minimal         | proper (server)     |

---

# 🧠 **Key Insight**

With MCP your LLM app becomes **interoperable** — our new vocabulary gem again — meaning it can work with multiple tool sources without rebuilding itself.

---





---

# **1. Pre-built tools vs Custom tools — What changes?**

### **Case A: Prebuilt LangChain Tools (No MCP)**

Examples:

* DuckDuckGoSearchTool
* SQLDatabaseTool
* VectorStoreRetrieverTool
* WikipediaUtility
* CalculatorTool

These are **local Python tool classes** baked directly into your app.
They do **not** use MCP.

Used like:

```python
llm.bind_tools([DuckDuckGoSearchTool()])
```

---

### **Case B: MCP Tools (Prebuilt or Custom)**

These are tools exposed by **external servers** (local or remote), for example:

* GitHub MCP Server
* Google Drive MCP Server
* Slack MCP Server
* Calendar MCP Server
* Math MCP Server (custom)
* CRM MCP Server (enterprise)
* Finance MCP Server (cloud)
* RAG/Vector MCP Server

These are accessed via:

```python
tools = await mcp_client.get_tools()
llm.bind_tools(tools)
```

---

# **2. So what does MCP add?**

It adds a **standard protocol** where tools can be:

✓ remote
✓ versioned
✓ auto-discovered
✓ shared between many apps
✓ updated independently
✓ polyglot (Node, Go, Rust, Python, etc)

This can’t be done with classic LangChain tools.

---

# **3. When MCP makes sense**

MCP is most valuable when tools are:

**✓ remote** (e.g., Google Calendar, CRM, ERP, Jira)
**✓ shared by many agents** (avoid rewriting APIs)
**✓ update frequently** (API versioning/hardening)
**✓ owned by other teams** (backend team builds server)

This matches enterprise patterns.

---

# **4. When MCP is overkill**

If tools are:

**✓ simple local helpers**
(e.g., `add()`, `base64_encode()`, `parse_date()`)

Using MCP adds no value — local Python tools are easier.

---

# **5. Blended Model — Works together**

The cool part: LangGraph can bind **both at the same time**:

```python
local_tools  = [DuckDuckGoSearchTool()]
mcp_tools    = await client.get_tools()

llm_with_tools = llm.bind_tools(local_tools + mcp_tools)
```

So you don’t choose one or the other — you combine both.

---

# **6. One more subtle point (important)**

Most pre-built LangChain tools **could be converted** into MCP servers in the future.

LangChain + LangGraph + Smith are trending toward:

> **pre-built tools become MCP servers**

Because it solves N×M explosion for multi-agent architectures.

---

# **7. Real Production Implication**

Your question implies this key distinction:

> “Is MCP only for custom tools we write?”

**Answer:**
No — MCP is a **delivery protocol**, not a “custom tool mechanism”.
Both prebuilt and custom tools can be delivered via MCP.

---

# **8. Example where MCP beats classic tools**

Take Google Calendar:

### Without MCP

Every app must:

* OAuth sign-in
* manage refresh tokens
* call Google API
* parse responses
* sanitize events
* handle rate limits
* handle timezones
* re-deploy on scope changes

### With MCP

Apps just do:

```
schedule meeting at 4pm
```

Server handles all the rest.

---

# **9. Where the industry is heading**

OpenAI, Anthropic, LangChain, ModelContextProtocol group are aligned toward:

* **Agents plug into enterprise APIs via MCP bridges**
* **LLMs not talking directly to APIs**
* **Backend teams own servers**
* **Frontend agents remain thin**

Think **USB** for tools instead of soldering wires.

---

# **10. Final clarification in one line**

> MCP is not about “custom vs prebuilt tools”,
> it’s about “embedded tools” vs “external standardized tool servers”.

-